In [1]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [4]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data"
ampl.read(mod_path)

# Assign set data

In [31]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [29]:
DFFR_PRICE = [1,2,3,4]
assign_set_data("DFFR_PRICE",DFFR_PRICE)
DA_PRICE = [1,2,3,4]
assign_set_data("DFFR_PRICE",DA_PRICE)
INTERVALS = [1,2]
df = assign_set_data("INTERVALS",INTERVALS)

# Assign parameter data

In [7]:
price_R = [1,2,4,5]
p_R = [.2,.3,.3,.1]
data = np.array([price_R,p_R]).transpose()
df = pd.DataFrame(data,columns = ["price_R","p_R"], index = DFFR_PRICE)
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)


In [8]:
# Example of getting parameter values
p = ampl.getParameter("price_R")
p.getValues().toPandas()

,price_R
1.0,1.0
2.0,2.0
3.0,4.0
4.0,5.0


In [25]:
# Assigning a multi-indexed parameter 
# (This parameter is not used in the model yet)
idx = pd.MultiIndex.from_product([DA_PRICE,INTERVALS],
                           names = ["DA_PRICE","INTERVALS"])
p_DA = np.ones(len(idx))
df = pd.DataFrame(p_DA, index = idx, columns=["p_DA"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

# Solve the model

In [22]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
settings = {'solver': 'cplexamp', 'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [23]:
ampl.solve()

CPLEX 12.5.1.0: mipdisplay = 2
mipgap = .005

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
CPLEX 12.5.1.0: QP Hessian is not positive semi-definite.
0 MIP simplex iterations
0 branch-and-bound nodes
No basis.


In [24]:
# Example of getting solution
p = ampl.getVariable("q_R")
p.getValues().toPandas()

,q_R.val
0,0.0
